In [2]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt

plt.style.use("ggplot")
pd.set_option("display.max_columns", 30)
pd.set_option("display.max_colwidth", 100)
pd.set_option("display.precision", 3)

CSV_PATH = r"./magic.csv"

df = pd.read_csv(CSV_PATH)
df.columns

Index(['routablemask_link', 'routablemask_link/_text',
       'routablemask_link/_title', 'routablemask_link_numbers',
       'routablemask_content', 'imagebadge_value', 'imagebadge_value_numbers',
       'routable_link', 'routable_link/_text', 'routable_link/_title',
       'routable_link_numbers', 'listingtype_value', 'pricelarge_value',
       'pricelarge_value_prices', 'propertyinfo_value',
       'propertyinfo_value_numbers', 'fineprint_value',
       'fineprint_value_numbers', 'tozcount_number', 'tozfresh_value',
       'tablegrouped_values', 'tablegrouped_values_prices', '_PAGE_NUMBER'],
      dtype='object')

In [4]:
df.head().T

,0,1,2,3,4
routablemask_link,"http://www.zillow.com/b/2-Ellwood-St-New-York-NY/40.85899,-73.92814_ll/","http://www.zillow.com/b/603-W-140th-St-New-York-NY/40.82376,-73.95335_ll/",http://www.zillow.com/homedetails/9-E-129th-St-1-New-York-NY-10035/2100761096_zpid/,http://www.zillow.com/homedetails/710-Riverside-Dr-APT-2C-New-York-NY-10031/124451755_zpid/,http://www.zillow.com/homedetails/413-E-84th-St-APT-8-New-York-NY-10028/2100761260_zpid/
routablemask_link/_text,5 photos,10 photos,NaN,9 photos,5 photos
routablemask_link/_title,NaN,NaN,NaN,NaN,NaN
routablemask_link_numbers,5.0,10.0,NaN,9.0,5.0
routablemask_content,NaN,NaN,NaN,NaN,NaN
imagebadge_value,5 photos,10 photos,NaN,9 photos,5 photos
imagebadge_value_numbers,5.0,10.0,NaN,9.0,5.0
routable_link,"http://www.zillow.com/b/2-Ellwood-St-New-York-NY/40.85899,-73.92814_ll/","http://www.zillow.com/b/603-W-140th-St-New-York-NY/40.82376,-73.95335_ll/",http://www.zillow.com/homedetails/9-E-129th-St-1-New-York-NY-10035/2100761096_zpid/,http://www.zillow.com/homedetails/710-Riverside-Dr-APT-2C-New-York-NY-10031/124451755_zpid/,http://www.zillow.com/homedetails/413-E-84th-St-APT-8-New-York-NY-10028/2100761260_zpid/
routable_link/_text,2 Ellwood St,603 W 140th St,"9 E 129th St # 1, New York, NY10035","710 Riverside Dr APT 2C, New York, NY10031","413 E 84th St APT 8, New York, NY10028"
routable_link/_title,"2 Ellwood St APT 5H, New York, NY Real Estate","603 W 140th St APT 44, New York, NY Real Estate","9 E 129th St # 1, New York, NY Real Estate","710 Riverside Dr APT 2C, New York, NY Real Estate","413 E 84th St APT 8, New York, NY Real Estate"


In [5]:
mu = df[df['listingtype_value'].str.contains('Apartments For')]
su = df[df['listingtype_value'].str.contains('Apartment For')]

In [6]:
su['propertyinfo_value']

2                    1 bd • 1 ba
3      2 bds • 2 ba • 1,016 sqft
4                  Studio • 1 ba
5                   2 bds • 1 ba
7                   2 bds • 1 ba
                 ...            
490                  1 bd • 1 ba
492                  1 bd • 1 ba
496                Studio • 1 ba
497     Studio • 1 ba • 550 sqft
499                  1 bd • 1 ba
Name: propertyinfo_value, Length: 339, dtype: object

In [9]:
len(su[~(su['propertyinfo_value'].str.contains('Studio') | su['propertyinfo_value'].str.contains('bd'))])

0

In [10]:
len(su[~(su['propertyinfo_value'].str.contains('bd'))])

122

In [13]:
su[~(su['propertyinfo_value'].str.contains('ba'))]['propertyinfo_value']

84     Studio
106      1 bd
150    Studio
225      1 bd
313    Studio
331      1 bd
Name: propertyinfo_value, dtype: object

In [14]:
no_baths = su[~(su['propertyinfo_value'].str.contains('ba'))]

In [15]:
sucln = su[~su.index.isin(no_baths.index)]

In [18]:
def parse_info(row):
  if not 'sqft' in row:
    br, ba = row.split(' ')[:2]
    sqft = np.nan
  else:
    br, ba, sqft = row.split('.')[:3]
  return pd.Series({'Beds': br, 'Baths':ba, 'Sqrt':sqft})

attr = sucln['propertyinfo_value'].apply(parse_info)

ValueError: not enough values to unpack (expected 3, got 1)